In [29]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

import sklearn
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, LinearRegression, HuberRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

import itertools
import pickle
from tqdm import tqdm

from datetime import datetime
from datetime import timedelta

import optuna
import pickle
import joblib
import xgboost

n_jobs = 20

In [2]:
df_train = pd.read_csv('/storage/kubrick/vkoshkina/data/train_classic_full.csv', sep=',', decimal='.', parse_dates=['photo_datetime', 'radiation_datetime'])
df_test = pd.read_csv('/storage/kubrick/vkoshkina/data/test_classic_full.csv', sep=',', decimal='.', parse_dates=['photo_datetime', 'radiation_datetime'])
#df_train.shape, df_test.shape
df_train['sin_sun'] = np.sin(np.radians(df_train['sun_altitude'].values))
df_test['sin_sun'] = np.sin(np.radians(df_test['sun_altitude'].values))
df_train['date_hour'] = pd.to_datetime(df_train['photo_datetime'].dt.date) + \
           pd.to_timedelta(df_train['photo_datetime'].dt.hour, unit='hours')
df_test['date_hour'] = pd.to_datetime(df_test['photo_datetime'].dt.date) + \
           pd.to_timedelta(df_test['photo_datetime'].dt.hour, unit='hours')
df_full = pd.concat((df_train, df_test))
df_full['date_hour'] = pd.to_datetime(df_full['photo_datetime'].dt.date) + \
           pd.to_timedelta(df_full['photo_datetime'].dt.hour, unit='hours')
df_full.head()

,photo_name,photo_datetime,CM3up[W/m2],CG3up[W/m2],CM3down[W/m2],CG3down[W/m2],radiation_datetime,feature0,feature1,feature2,...,feature159,feature160,feature161,date-hour,datetime_UTC,lat,lon,sun_altitude,sin_sun,date_hour
0,img-2014-09-17T10-00-43devID1.jpg,2014-09-17 10:00:43,158.833993,-91.263207,39.045110,-2.084933,2014-09-17 10:00:44,97.094520,1751.681381,255.0,...,0.538882,0.722970,0.973412,2014-09-17-10,2014-09-17 10:00:43.290268,66.583405,-29.273292,14.318959,0.247320,2014-09-17 10:00:00
1,img-2014-09-17T10-00-44devID2.jpg,2014-09-17 10:00:44,158.833993,-91.263207,39.045110,-2.084933,2014-09-17 10:00:44,107.922526,1848.315610,255.0,...,0.553206,0.722807,0.973412,2014-09-17-10,2014-09-17 10:00:43.290268,66.583405,-29.273292,14.318959,0.247320,2014-09-17 10:00:00
2,img-2014-09-17T10-01-43devID1.jpg,2014-09-17 10:01:43,149.357024,-88.420117,34.590935,0.189539,2014-09-17 10:01:44,96.720268,1685.916556,255.0,...,0.538814,0.703659,0.973412,2014-09-17-10,2014-09-17 10:01:43.301701,66.583967,-29.276475,14.403886,0.248756,2014-09-17 10:00:00
3,img-2014-09-17T10-01-44devID2.jpg,2014-09-17 10:01:44,149.357024,-88.420117,34.590935,0.189539,2014-09-17 10:01:44,107.917577,1830.592868,255.0,...,0.554802,0.722170,0.973412,2014-09-17-10,2014-09-17 10:01:43.301701,66.583967,-29.276475,14.403886,0.248756,2014-09-17 10:00:00
4,img-2014-09-17T10-02-43devID1.jpg,2014-09-17 10:02:43,143.102225,-88.135807,36.391559,0.473848,2014-09-17 10:02:44,96.890459,1671.656306,255.0,...,0.543301,0.702826,0.973412,2014-09-17-10,2014-09-17 10:02:43.303133,66.584686,-29.280270,14.488320,0.250183,2014-09-17 10:00:00


In [3]:
def create_Xy(df_train):
    feature_columns = [c for c in df_train.columns if c.startswith('f') or c=='sin_sun']    
    ytr = df_train['CM3up[W/m2]'].values
    Xtr = df_train[feature_columns].values
    #print('dataset_features: ' + str(Xtr.shape) + ', target: ' + str(ytr.shape))
    return Xtr, ytr

In [4]:
Xtr, ytr = create_Xy(df_train)
Xtest, ytest = create_Xy(df_test)

In [ ]:
(ytr[:,np.newaxis]>=bounds[:-1,np.newaxis].T) & (ytr[:,np.newaxis]<=bounds[1:,np.newaxis].T) [1]

In [ ]:
perc = np.percentile(ytr, np.arange(0, 101))
di = perc[1:] - perc[:-1]
wi = (1/di)/np.mean(1/di)
weights_with_alfa = (wi-1)*alfa+1
weight_index = np.where(((ytr[:,np.newaxis]>=di[:-1,np.newaxis].T) & (ytr[:,np.newaxis]<=di[1:,np.newaxis].T) ) )  [1]
complex_weights = wi[weight_index]
len(weight_index), len(ytr), len(bounds), len( d_i)

In [6]:
alfa = 0.5

In [23]:
# bounds = np.percentile(ytr, np.arange(0, 101))
# d_i = bounds[1:] - bounds[:-1]
# weights_classic =  weights_classic = (1/d_i)/np.mean(1/d_i)
#weight_index = np.where((ytr[:,np.newaxis]>=d_i[:-1,np.newaxis].T) & (ytr[:,np.newaxis]<d_i[1:,np.newaxis].T)) [1]
#weight_index = np.where((ytr[:,np.newaxis]>=bounds[:-1,np.newaxis].T) & (ytr[:,np.newaxis]<=bounds[1:,np.newaxis].T)) [1]
#weight_index_2_way = np.searchsorted(d_i, ytest, side='right')
# weight_index = np.where(((ytest[:,np.newaxis]>=di[:-1,np.newaxis].T) & (ytest[:,np.newaxis]<=di[1:,np.newaxis].T) ) )  [1]

perc = np.percentile(ytr, np.arange(0, 101))
di = perc[1:] - perc[:-1]
wi = (1/di)/np.mean(1/di)
weights_with_alfa = (wi-1)*alfa+1
weight_index = np.searchsorted(di, ytr, side='left') - 1
complex_weights = weights_with_alfa[weight_index]
len(weight_index), len(ytr), len(perc), len( di)

(1041734, 1041734, 101, 100)

In [22]:
weight_index.min(), weight_index.max()

(43, 99)

In [25]:
def calculate_weights_with_alfa(target: np.ndarray, alfa: float):
    perc = np.percentile(target, np.arange(0, 101))
    di = perc[1:] - perc[:-1]
    wi = (1/di)/np.mean(1/di)
    weights_with_alfa = (wi-1)*alfa+1
    weight_index = np.searchsorted(di, target, side='left') - 1
    complex_weights = weights_with_alfa[weight_index]
    return complex_weights

sample_weight = calculate_weights_with_alfa(ytr, 0.5)




In [11]:
#model = xgboost.XGBRegressor(n_estimators = 100, max_depth = 15,n_jobs = 20, objective='reg:squarederror')
model = xgboost.XGBRegressor(n_jobs = 20, objective='reg:squarederror')
model.fit(Xtr, ytr, sample_weight = complex_weights)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100,
             n_jobs=20, num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0, reg_lambda=1, ...)

In [12]:
ytest_pred = model.predict(Xtest)
rmse_value = np.sqrt(mean_squared_error(ytest_pred, ytest))
print('Prediction: %.3f' % rmse_value)

Prediction: 65.310


# поиграться с регуляризацией
+взвешивание весов и взвешенный лосс
+ random forest regressor с взвешиванием



In [ ]:
bounds = np.percentile(ytr, np.arange(0, 101))
d_i = bounds[1:] - bounds[:-1]
weights_classic = 100/(d_i*(np.sum(1/d_i)))
weights_with_alfa = (weights_classic-1)*alfa+1
weight_index = np.searchsorted(np.percentile(ytr, np.arange(0, 101)), ytr)#, side='right')
simple_weights = weights[weight_index]
simple_weights

In [ ]:
    sample_weight: Optional[Any] = None, в fit
reg_alpha : Optional[float]
        L1 regularization term on weights (xgb's alpha).

по ошибке сети не надо
перевзесить по частотности таргетом, сделать гиперпараметром
на тренировочной вычислить веса и использовать

In [37]:
# 1. Define an objective function to be maximized.
def objective(trial):
    # 2. Suggest values for the hyperparameters using a trial object.
    gbr_n_estimators = trial.suggest_int('gbr_n_estimators', 100, 1500, log=True) # от первого до второго числа перебираем количество деревьев
    gbr_max_depth = trial.suggest_int('gbr_max_depth', 5, 30, log=True) # от первого до второго числа перебираем максимальную глубину 
    alfa_range = trial.suggest_float('alfa', 0.0001, 1, log=True) # от первого до второго числа перебираем alfa
    regressor_obj = xgboost.XGBRegressor(max_depth=gbr_max_depth, n_estimators=gbr_n_estimators, n_jobs = n_jobs, objective='reg:squarederror')
    train, test = train_test_split(df_full['date_hour'].unique(),
                                       test_size=0.05,
                                       train_size=0.20)
    df_train = df_full[df_full['date_hour'].isin(train)]
    df_test = df_full[df_full['date_hour'].isin(test)]
    Xtr, ytr = create_Xy(df_train)
    Xtest, ytest = create_Xy(df_test)
    sample_weight = calculate_weights_with_alfa(ytr, alfa_range)
    regressor_obj.fit(Xtr, ytr, sample_weight = sample_weight)
    ytest_pred = regressor_obj.predict(Xtest)
    mae_value = (mean_absolute_error(ytest_pred, ytest))
    if trial.should_prune():
        raise optuna.TrialPruned()
    return mae_value

In [36]:
if __name__ == "__main__":
    #err_fname = './logs/hpo_richards_gamma.err'
    db_fname = 'Cubrick_predict_sun_swrad.db'
    study_name = 'Cubrick_predict_sun_swrad_gbr_with_weights'

    try:
        study = optuna.load_study(study_name=study_name,
                                  storage="sqlite:///%s" % db_fname,
                                  sampler=optuna.samplers.TPESampler())
    except:
        study = optuna.create_study(storage='sqlite:///%s' % db_fname,
                                    study_name=study_name,
                                    sampler=optuna.samplers.TPESampler(),
                                    direction="minimize")

    study.optimize(objective, n_trials=300, gc_after_trial=True)

[W 2022-06-25 02:13:12,145] Trial 3 failed because of the following error: UnboundLocalError("local variable 'ytr' referenced before assignment")
Traceback (most recent call last):
  File "/storage/kubrick/mborisov/conda/lib/python3.9/site-packages/optuna/study/_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_9275/4170028161.py", line 7, in objective
    sample_weight = calculate_weights_with_alfa(ytr, 0.5)
UnboundLocalError: local variable 'ytr' referenced before assignment


UnboundLocalError: local variable 'ytr' referenced before assignment